In [9]:
import cv2
import numpy as np
import os
os.environ["OPENCV_VIDEOIO_PRIORITY_MSMF"] = "0"
import time
import io


cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# cv2.namedWindow('frame', flags=cv2.CV_GUI_EXPANDED)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
parameters =  cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)
a = 0
b = 0
c = 0
d = 0
new_w = 846
new_h = 846
full_w = 1920
full_h = 1080

def dist_func(left, right):
    scale = np.sqrt((right[0]-left[0])**2+(right[1]-left[1])**2)/300.0
    if (right[0]-left[0])!=0:
        theta = np.arctan((right[1]-left[1])/(right[0]-left[0]))+np.pi/4
    else:
        theta = np.pi/2
    x = scale*np.sqrt(2)*80/np.sqrt(np.tan(theta)**2+1)
    y = np.tan(theta)*x
    return (x, y)
    

while(cap.isOpened()):
    try:
        ret, frame = cap.read()
        locs_and_ids = []
        image = frame.copy()
        corners, ids, rejectedCandidates = detector.detectMarkers(image)
    
        if len(corners) > 0:
            # flatten the ArUco IDs list
            ids = ids.flatten()
            # loop over the detected ArUCo corners
            for (markerCorner, markerID) in zip(corners, ids):
                corners = markerCorner.reshape((4, 2))
                (topLeft, topRight, bottomRight, bottomLeft) = corners
                topRight = (int(topRight[0]), int(topRight[1]))
                bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
                bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
                topLeft = (int(topLeft[0]), int(topLeft[1]))
                if markerID==1:
                    temp_a_dist = dist_func(topLeft, topRight)
                    a = (topLeft[0]-int(temp_a_dist[0]), topLeft[1]-int(temp_a_dist[1]))
                elif markerID==2:
                    # temp_b_dist = int((topRight[0]-topLeft[0])/300.0*80)
                    temp_b_dist = dist_func(topLeft, topRight)
                    b = (topRight[0]+int(temp_b_dist[0]), topRight[1]-int(temp_b_dist[1]))
                elif markerID==3:
                    # temp_c_dist = int((bottomRight[0]-bottomLeft[0])/300.0*80)
                    temp_c_dist = dist_func(bottomLeft, bottomRight)
                    c = (bottomLeft[0]-int(temp_c_dist[0]), bottomLeft[1]+int(temp_c_dist[1]))
                elif markerID==4:
                    # temp_d_dist = int((bottomRight[0]-bottomLeft[0])/300.0*80)
                    temp_d_dist = dist_func(bottomLeft, bottomRight)
                    d = (bottomRight[0]+int(temp_d_dist[0]), bottomRight[1]+int(temp_d_dist[1]))
                cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
                cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
                cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
                cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
        if a != 0 and b != 0 and c != 0 and d != 0:
            src = np.float32([a,c,d,b])
        else:
            src = np.float32([((full_w-new_w)/2, (full_h-new_h)/2), 
                              ((full_w-new_w)/2, (full_h-new_h)/2+new_h), 
                              ((full_w-new_w)/2+new_w,  (full_h-new_h)/2+new_h), 
                              ((full_w-new_w)/2+new_w,  (full_h-new_h)/2)])
        
        dst = np.float32([(0, 0),
                          (0, new_h),
                          (new_w, new_h),
                          (new_w, 0)])
        M = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(frame, M, (new_w, new_h))
        border = cv2.copyMakeBorder(
            warped,
            top=int((full_h-new_h)/2),
            bottom=int((full_h-new_h)/2),
            left=int((full_w-new_w)/2),
            right=int((full_w-new_w)/2),
            borderType=cv2.BORDER_CONSTANT,
            value=[0, 0, 0]
        )
        im_rgb = cv2.cvtColor(border, cv2.COLOR_BGR2RGB)
        masked = mask_picture_frame(image, src, image)
        cv2.imshow('im_rgb', im_rgb)
        cv2.imshow('frame', image)
        cv2.imshow('masked', masked)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except:
        break
cv2.destroyAllWindows()
for i in range(5):
    cv2.waitKey(1)
cap.release()

In [7]:
masked = mask_picture_frame(image, src, image)

ValueError: shape mismatch: value array of shape (1080,1920,3) could not be broadcast to indexing result of shape (717409,3)

In [ ]:
def mask_picture_frame(base_image, corners, fill_image):
    """
    Masks the area inside the trapezoid defined by corners with pink pixels.
    
    Parameters:
        image (numpy.ndarray): The input image.
        corners (list of tuple): Four corner points (x, y) of the trapezoid in clockwise order.
        
    Returns:
        numpy.ndarray: The modified image with the trapezoid filled with pink pixels.
    """
    # Define the trapezoid as a polygon
    pts = np.array(corners, dtype=np.int32)

    # Create a mask the same size as the image
    mask = np.zeros_like(image, dtype=np.uint8)

    # Fill the trapezoid in the mask with white (255, 255, 255)
    cv2.fillPoly(mask, [pts], (255, 255, 255))

    # Create the pink color (BGR)
    # pink_color = (255, 105, 180)  # Bright pink in BGR

    # Use the mask to replace the trapezoid area with pink
    # image[np.where((mask == 255).all(axis=2))] = pink_color

    condition = np.stack((mask,) * 3, axis=-1) > 0

    # Prepare foreground and background images
    fg_image = fill_image.copy()
    bg_image = base_image.copy()

    # Blend the images using the condition mask
    output_image = np.where(condition, fg_image, bg_image)

    return output_image

In [8]:
def mask_picture_frame(image, corners, second):
    """
    Masks the area inside the trapezoid defined by corners with pink pixels.
    
    Parameters:
        image (numpy.ndarray): The input image.
        corners (list of tuple): Four corner points (x, y) of the trapezoid in any order.
        
    Returns:
        numpy.ndarray: The modified image with the trapezoid filled with pink pixels.
    """
    # Ensure corners are a numpy array
    corners = np.array(corners, dtype=np.float32)

    # Calculate the center of the corners
    center = np.mean(corners, axis=0)

    # Sort corners based on their angle from the center
    def angle_from_center(point):
        return np.arctan2(point[1] - center[1], point[0] - center[0])
    
    sorted_corners = sorted(corners, key=angle_from_center)

    # Convert sorted corners back to numpy array
    sorted_corners = np.array(sorted_corners, dtype=np.int32)

    # Create a mask the same size as the image
    mask = np.zeros_like(image, dtype=np.uint8)

    # Fill the trapezoid in the mask with white (255, 255, 255)
    cv2.fillPoly(mask, [sorted_corners], (255, 255, 255))

    # Define the pink color (BGR)
    pink_color = (255, 105, 180)  # Bright pink in BGR

    # Use the mask to replace the trapezoid area with pink
    image[np.where((mask == 255).all(axis=2))] = second[np.where((mask == 255).all(axis=2))]

    return image